# GPT-2 Text Prediction with OpenVINO

This notebook shows a text prediction with OpenVINO. We use the [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) model, which is a part of the Generative Pre-trained Transformer (GPT) family. GPT-2 is pre-trained on a large corpus of English text using unsupervised training. The model is available from [HuggingFace](https://huggingface.co/gpt2). GPT-2 displays a broad set of capabilities, including the ability to generate conditional synthetic text samples of unprecedented quality, where we prime the model with an input and have it generate a lengthy continuation.

The following image illustrates complete demo pipeline used for this scenario:

![image2](https://user-images.githubusercontent.com/91228207/163990722-d2713ede-921e-4594-8b00-8b5c1a4d73b5.jpeg)

Model input is tokenized text, which serves as initial condition for generation, then logits from model inference result should be obtained and token with the highest probability is selected using top-k sampling strategy and joined to input sequence. The procedure repeats until end of sequence token will be recived or specified maximul length will be reached. After that, decoding token ids to text using tokenized should be applied.


## The model


In [98]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
pt_model = GPT2LMHeadModel.from_pretrained('gpt2')

## Convert GPT-2 to OpenVINO IR

![conversion_pipeline](https://user-images.githubusercontent.com/29454499/211261803-784d4791-15cb-4aea-8795-0969dfbb8291.png)

For starting work with GPT2 model using OpenVINO, model should be converted to OpenVINO Intermediate Represenation (IR) format. HuggingFace provided gpt2 model is PyTorch model, which supported in OpenVINO via conversion to ONNX. We will use HuggingFace transformers library capabilities for export model to ONNX. `transformers.onnx.export` accept preprocessing function for input sample generation (tokenizer in our case), instance of model, ONNX export configuration, ONNX opset version for export and output path. More information about transformers export to ONNX can be found in HuggingFace [documentation](https://huggingface.co/docs/transformers/serialization).

While ONNX models are directly supported by OpenVINO runtime, it can be useful to convert them to IR format to take advantage of OpenVINO optimization tools and features.
`mo.convert_model` python function can be used for converting model using [OpenVINO Model Optimizer](https://docs.openvino.ai/latest/openvino_docs_MO_DG_Python_API.html). The function returns instance of OpenVINO Model class, which is ready to use in Python interface but can also be serialized to OpenVINO IR format for future execution using `openvino.runtime.serialize`. In our case, `compress_to_fp16` parameter is enabled for compression model weights to fp16 precision and also specified dynamic input shapes with possible shape range (from 1 token to maximum length defined in our processing function) for optimization of memory consumption.

In [99]:
from pathlib import Path
from openvino.runtime import serialize
from openvino.tools import mo
from transformers.onnx import export, FeaturesManager


# define path for saving onnx model
onnx_path = Path("model/gpt2.onnx")
onnx_path.parent.mkdir(exist_ok=True)

# define path for saving openvino model
model_path = onnx_path.with_suffix(".xml")

# get model onnx config function for output feature format casual-lm
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(pt_model, feature='causal-lm')

# fill onnx config based on pytorch model config
onnx_config = model_onnx_config(pt_model.config)

# convert model to onnx
onnx_inputs, onnx_outputs = export(tokenizer, pt_model, onnx_config, onnx_config.default_onnx_opset, onnx_path)

# convert model to openvino
ov_model = mo.convert_model(onnx_path, compress_to_fp16=True, input="input_ids[1,1..128],attention_mask[1,1..128]")

# serialize openvino model
serialize(ov_model, str(model_path))

### Load the model

We start by building an Inference Engine object. Then we read the network architecture and model weights from the .xml and .bin files, respectively. Finally, we compile the model for the desired device. Because we use the dynamic shapes feature, which is only available on CPU, we must use `CPU` for the device. Dynamic shapes support on GPU is coming soon.

Since the text recognition model has a dynamic input shape, you cannot directly switch device to `GPU` for inference on integrated or discrete Intel GPUs. In order to run inference on iGPU or dGPU with this model, you will need to resize the inputs to this model to use a fixed size and then try running the inference on `GPU` device.

In [ ]:
from openvino.runtime import Core

# initialize inference engine
ie_core = Core()

# read the model and corresponding weights from file
model = ie_core.read_model(model_path)

# compile the model for CPU devices
compiled_model = ie_core.compile_model(model=model, device_name="CPU")

# get output tensors
output_key = compiled_model.output(0)

Input keys are the names of the input nodes and output keys contain names of the output nodes of the network. In the case of GPT-2, we have `batch size` and `sequence length` as inputs and `batch size`, `sequence length` and `vocab size` as outputs.

## Pre-Processing

NLP models often take a list of tokens as a standard input. A token is a single word mapped to an integer. To provide the proper input, we use a vocabulary file to handle the mapping. So first let's load the vocabulary file.

## Define tokenization

In [ ]:
# this function converts text to tokens
def tokenize(text):
    """
    tokenize input text using GPT2 tokenizer
    
    Parameters:
      text, str - input text
    Returns:
      input_ids - np.array with input token ids
      attention_mask - np.array with 0 in place, where should be padding and 1 for places where original tokens are located, represents attention mask for model 
    """
    
    inputs = tokenizer(text, return_tensors="np")
    return inputs["input_ids"], inputs["attention_mask"]

`eos_token` is special token, which means that generation is finished. We store the index of this token in order to use this index as padding at later stage.

In [ ]:
eos_token_id = tokenizer.eos_token_id

### Define Softmax layer
A softmax function is used to convert top-k logits into a probability distribution. 

In [ ]:
import numpy as np


def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    summation = e_x.sum(axis=-1, keepdims=True)
    return e_x / summation

### Set the minimum sequence length  
If the minimum sequence length is not reached, the following code will reduce the probability of the `eos` token occurring. This continues the process of generating the next words.

In [ ]:
def process_logits(cur_length, scores, eos_token_id, min_length=0):
    """
    reduce probability for padded indicies
    
    Parameters:
      cur_length - current length of input sequence
      scores - model output logits
      eos_token_id - index of end of string token in model vocab
      min_length - minimum length for appling postprocessing
    """
    if cur_length < min_length:
        scores[:, eos_token_id] = -float("inf")
    return scores

### Top-K sampling
In Top-K sampling, we filter the K most likely next words and redistribute the probability mass among only those K next words. 

In [ ]:
def get_top_k_logits(scores, top_k):
    """
    perform top-k sampling
    
    Parameters:
      scores - model output logits
      top_k - number of elements with highest probability to select
    """
    filter_value = -float("inf")
    top_k = min(max(top_k, 1), scores.shape[-1])
    top_k_scores = -np.sort(-scores)[:, :top_k]
    indices_to_remove = scores < np.min(top_k_scores)
    filtred_scores = np.ma.array(scores, mask=indices_to_remove,
                                 fill_value=filter_value).filled()
    return filtred_scores

### Main Processing Function
Generating the predicted sequence.

In [ ]:
def generate_sequence(input_ids, attention_mask, max_sequence_length=128,
                      eos_token_id=eos_token_id, dynamic_shapes=True):
    """
    text prediction cycle.

    Parameters:
      input_ids: tokenized input ids for model
      attention_mask: attention mask for model
      max_sequence_length: maximum sequence length for stop iteration
      eos_token_ids: end of sequence index from vocab
      dynamic_shapes: use dynamic shapes for inference or pad model input to max_sequece_length
    Returns:
      predicted token ids sequence
    """
    while True:
        cur_input_len = len(input_ids[0])
        if not dynamic_shapes:
            pad_len = max_sequence_length - cur_input_len
            model_input_ids = np.concatenate((input_ids, [[eos_token_id] * pad_len]), axis=-1)
            model_input_attention_mask = np.concatenate((attention_mask, [[0] * pad_len]), axis=-1)
        else:
            model_input_ids = input_ids
            model_input_attention_mask = attention_mask
        outputs = compiled_model({"input_ids": model_input_ids, "attention_mask": model_input_attention_mask})[output_key]
        next_token_logits = outputs[:, cur_input_len - 1, :]
        # pre-process distribution
        next_token_scores = process_logits(cur_input_len,
                                           next_token_logits, eos_token_id)
        top_k = 20
        next_token_scores = get_top_k_logits(next_token_scores, top_k)
        # get next token id
        probs = softmax(next_token_scores)
        next_tokens = np.random.choice(probs.shape[-1], 1,
                                       p=probs[0], replace=True)
        # break the loop if max length or end of text token is reached
        if cur_input_len == max_sequence_length or next_tokens == eos_token_id:
            break
        else:
            input_ids = np.concatenate((input_ids, [next_tokens]), axis=-1)
            attention_mask = np.concatenate((attention_mask, [[1] * len(next_tokens)]), axis=-1)
    return input_ids

# Run
The `text` variable below is the input used to generate a predicted sequence.

In [ ]:
import time
text = "Deep learning is a type of machine learning that uses neural networks"
input_ids, attention_mask = tokenize(text)

start = time.perf_counter()
output_ids = generate_sequence(input_ids, attention_mask)
end = time.perf_counter()
output_text = " "
# Convert IDs to words and make the sentence from it
for i in output_ids[0]:
    output_text += tokenizer.convert_tokens_to_string(tokenizer._convert_id_to_token(i))
print(f"Generation took {end - start:.3f} ms")
print("Input Text: ", text)
print()
print(f"Predicted Sequence:{output_text}")

In [100]:
# define path for saving onnx model
onnx_path = Path("model/gpt2_with_past.onnx")

In [102]:
# define path for saving openvino model
model_path = onnx_path.with_suffix(".xml")

# get model onnx config function for output feature format casual-lm
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(pt_model, feature='causal-lm-with-past')

# fill onnx config based on pytorch model config
onnx_config = model_onnx_config(pt_model.config)

# convert model to onnx
onnx_inputs, onnx_outputs = export(tokenizer, pt_model, onnx_config, onnx_config.default_onnx_opset, onnx_path)

In [106]:
dummy_inputs = onnx_config.generate_dummy_inputs(tokenizer, framework="pt")

In [111]:
dummy_inputs["past_key_values"][0][0].shape

torch.Size([2, 12, 10, 64])

In [112]:
dummy_inputs["input_ids"].shape

torch.Size([2, 8])

In [114]:
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model='gpt2')

In [115]:
??generator

Signature:      generator(text_inputs, **kwargs)
Type:           TextGenerationPipeline
String form:    <transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f64f2429550>
File:           ~/work/notebooks_env/lib/python3.8/site-packages/transformers/pipelines/text_generation.py
Source:        
class TextGenerationPipeline(Pipeline):
    """
    Language generation pipeline using any `ModelWithLMHead`. This pipeline predicts the words that will follow a
    specified text prompt.

    This language generation pipeline can currently be loaded from [`pipeline`] using the following task identifier:
    `"text-generation"`.

    The models that this pipeline can use are models that have been trained with an autoregressive language modeling
    objective, which includes the uni-directional models in the library (e.g. gpt2). See the list of available models
    on [huggingface.co/models](https://huggingface.co/models?filter=text-generation).
    """

    # Prefix text to 

In [137]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from openvino.runtime import Core

core = Core()

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

compiled_model = core.compile_model("model/gpt2_with_past.onnx")



inputs = tokenizer(text, return_tensors="pt")
generated = tokenizer.encode(text)
context = inputs.input_ids
seq_len = context.shape[1]
attention_mask = inputs.attention_mask

feed_dict = {}
past_key_values = None
# past_key_values = [torch.zeros(1, 12, seq_len, 64) for i in range(24))

for i in range(30):
    output = model(context, attention_mask=attention_mask, past_key_values=past_key_values)
    past_key_values = output.past_key_values
    token = torch.argmax(output.logits[..., -1, :])

    context = token.unsqueeze(0)
    attention_mask = torch.ones(1, 1)
    print(past_key_values[0][0].shape)
   
    generated += [token.tolist()]


sequence = tokenizer.decode(generated)
sequence = sequence.split(".")[:-1]
print(sequence)

torch.Size([1, 12, 12, 64])
torch.Size([1, 12, 13, 64])
torch.Size([1, 12, 14, 64])
torch.Size([1, 12, 15, 64])
torch.Size([1, 12, 16, 64])
torch.Size([1, 12, 17, 64])
torch.Size([1, 12, 18, 64])
torch.Size([1, 12, 19, 64])
torch.Size([1, 12, 20, 64])
torch.Size([1, 12, 21, 64])
torch.Size([1, 12, 22, 64])
torch.Size([1, 12, 23, 64])
torch.Size([1, 12, 24, 64])
torch.Size([1, 12, 25, 64])
torch.Size([1, 12, 26, 64])
torch.Size([1, 12, 27, 64])
torch.Size([1, 12, 28, 64])
torch.Size([1, 12, 29, 64])
torch.Size([1, 12, 30, 64])
torch.Size([1, 12, 31, 64])
torch.Size([1, 12, 32, 64])
torch.Size([1, 12, 33, 64])
torch.Size([1, 12, 34, 64])
torch.Size([1, 12, 35, 64])
torch.Size([1, 12, 36, 64])
torch.Size([1, 12, 37, 64])
torch.Size([1, 12, 38, 64])
torch.Size([1, 12, 39, 64])
torch.Size([1, 12, 40, 64])
torch.Size([1, 12, 41, 64])
['Deep learning is a type of machine learning that uses neural networks to learn from data', '\n\nThe problem is that the data is not always accurate, and the a

In [ ]:
compiled_model

logits_out = compiled_model.output(0)
preset_outputs = [compiled_model.output(i) for i in range(1, len(compiled_model.outputs))]
feed_dict = {}
for i in range(len(preset_outputs)):
    feed_dict.update({
        f"past_key_values.{i}.key": torch.zeros(1, 12, seq_len, 64),
        f"past_key_values.{i}.value": torch.zeros(1, 12, seq_len, 64)
    })

feed_dict["input_ids"] = inputs.input_ids
feed_dict["attention_mask"] = inputs.attention_mask

In [138]:
compiled_model(feed_dict)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-138-be9c4f570bad>:1 in <module>                                                   │
│                                                                                                  │
│ /home/ea/work/notebooks_env/lib/python3.8/site-packages/openvino/runtime/ie_api.py:273 in        │
│ __call__                                                                                         │
│                                                                                                  │
│   270 │   │                                                                                      │
│   271 │   │   Take a look at `infer_new_request` for reference.                                  │
│   272 │   │   """                                                                                │
│ ❱ 273 │   │   return self.infer_new_request(inputs)                                              │
│   274                                                                                            │
│   275                                                                                            │
│   276 class AsyncInferQueue(AsyncInferQueueBase):                                                │
│                                                                                                  │
│ /home/ea/work/notebooks_env/lib/python3.8/site-packages/openvino/runtime/ie_api.py:266 in        │
│ infer_new_request                                                                                │
│                                                                                                  │
│   263 │   │   """                                                                                │
│   264 │   │   # It returns wrapped python InferReqeust and then call upon                        │
│   265 │   │   # overloaded functions of InferRequest class                                       │
│ ❱ 266 │   │   return self.create_infer_request().infer(inputs)                                   │
│   267 │                                                                                          │
│   268 │   def __call__(self, inputs: Optional[Union[dict, list]] = None) -> dict:                │
│   269 │   │   """Callable infer wrapper for CompiledModel.                                       │
│                                                                                                  │
│ /home/ea/work/notebooks_env/lib/python3.8/site-packages/openvino/runtime/ie_api.py:148 in infer  │
│                                                                                                  │
│   145 │   │   │   return super().infer({})                                                       │
│   146 │   │   # If inputs are dict, normalize dictionary and call infer method.                  │
│   147 │   │   elif isinstance(inputs, dict):                                                     │
│ ❱ 148 │   │   │   return super().infer(normalize_inputs(self, inputs))                           │
│   149 │   │   # If inputs are list or tuple, enumarate inputs and save them as dictionary.       │
│   150 │   │   # It is an extension of above branch with dict inputs.                             │
│   151 │   │   elif isinstance(inputs, (list, tuple)):                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Shape inference of Concat node with name present.0.key failed: Check 'T::merge_into(output_shape, 
input)' failed at src/core/shape_inference/include/concat_shape_inference.hpp:37:
While validating node 'v0::Concat present.0.key (past_key_values.0.key[0]:f32[?,12,?,64], 
/transformer/h.0/attn/Transpose_1[0]:f32[?,12,?,64]) -> (f32[?,12,?,64])' with friendly_name 'present.0.key':
Argument shapes are inconsistent; they must have the same rank, and m

In [140]:
past_key_values = ""
for i in range(12):
    past_key_values += f",past_key_values.{i}.key[1,12,{seq_len}..128,64],past_key_values.{i}.value[1,12,{seq_len}..128,64]"

ov_model_with_past = mo.convert_model("model/gpt2_with_past.onnx", compress_to_fp16=True, input=f"input_ids[1,1..{seq_len}],attention_mask[1,1..{seq_len}]"+past_key_values)

In [142]:
compiled_model = core.compile_model(ov_model_with_past)

In [143]:
compiled_model(feed_dict)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-143-be9c4f570bad>:1 in <module>                                                   │
│                                                                                                  │
│ /home/ea/work/notebooks_env/lib/python3.8/site-packages/openvino/runtime/ie_api.py:273 in        │
│ __call__                                                                                         │
│                                                                                                  │
│   270 │   │                                                                                      │
│   271 │   │   Take a look at `infer_new_request` for reference.                                  │
│   272 │   │   """                                                                                │
│ ❱ 273 │   │   return self.infer_new_request(inputs)                                              │
│   274                                                                                            │
│   275                                                                                            │
│   276 class AsyncInferQueue(AsyncInferQueueBase):                                                │
│                                                                                                  │
│ /home/ea/work/notebooks_env/lib/python3.8/site-packages/openvino/runtime/ie_api.py:266 in        │
│ infer_new_request                                                                                │
│                                                                                                  │
│   263 │   │   """                                                                                │
│   264 │   │   # It returns wrapped python InferReqeust and then call upon                        │
│   265 │   │   # overloaded functions of InferRequest class                                       │
│ ❱ 266 │   │   return self.create_infer_request().infer(inputs)                                   │
│   267 │                                                                                          │
│   268 │   def __call__(self, inputs: Optional[Union[dict, list]] = None) -> dict:                │
│   269 │   │   """Callable infer wrapper for CompiledModel.                                       │
│                                                                                                  │
│ /home/ea/work/notebooks_env/lib/python3.8/site-packages/openvino/runtime/ie_api.py:148 in infer  │
│                                                                                                  │
│   145 │   │   │   return super().infer({})                                                       │
│   146 │   │   # If inputs are dict, normalize dictionary and call infer method.                  │
│   147 │   │   elif isinstance(inputs, dict):                                                     │
│ ❱ 148 │   │   │   return super().infer(normalize_inputs(self, inputs))                           │
│   149 │   │   # If inputs are list or tuple, enumarate inputs and save them as dictionary.       │
│   150 │   │   # It is an extension of above branch with dict inputs.                             │
│   151 │   │   elif isinstance(inputs, (list, tuple)):                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Shape inference of Concat node with name present.0.key failed: Check 'T::merge_into(output_shape, 
input)' failed at src/core/shape_inference/include/concat_shape_inference.hpp:37:
While validating node 'v0::Concat present.0.key (past_key_values.0.key[0]:f32[1,12,12..128,64], 
/transformer/h.0/attn/Transpose_1[0]:f32[?,12,1..12,64]) -> (f32[1,12,13..140,64])' with friendly_name 
'present.0.key':
Argument shapes are inconsistent; they must have the

In [145]:
ov_model_with_past

<Model: 'torch_jit'
inputs[
<ConstOutput: names[input_ids] shape[1,1..12] type: i64>,
<ConstOutput: names[past_key_values.0.key] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.0.value] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.1.key] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.1.value] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.2.key] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.2.value] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.3.key] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.3.value] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.4.key] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.4.value] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.5.key] shape[1,12,12..128,64] type: f32>,
<ConstOutput: names[past_key_values.5.value] s